In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('magazines_rating.csv')
df.head()

,user_id,magazine_id,ratings
0,AH2IFH762VY5U,B00005N7P0,5
1,AOSFI0JEYU4XM,B00005N7P0,5
2,A3JPFWKS83R49V,B00005N7OJ,3
3,A19FKU6JZQ2ECJ,B00005N7OJ,5
4,A25MDGOMZ2GALN,B00005N7P0,5


## Popularity based Recommender system

In [3]:
num_rating_df = df.groupby('magazine_id').count()['ratings'].reset_index()
num_rating_df = num_rating_df.rename(columns={'ratings':'number_of_ratings'})
num_rating_df

,magazine_id,number_of_ratings
0,B00005N7NQ,116
1,B00005N7O3,5
2,B00005N7O4,6
3,B00005N7O6,64
4,B00005N7O9,23
...,...,...
2423,B01HI8V1AE,4
2424,B01HI8V1C2,10
2425,B01HI8V1I6,24
2426,B01HI8V1MC,14


In [14]:
avg_rating_df = df[["magazine_id","ratings"]].groupby('magazine_id').mean()['ratings'].reset_index()
avg_rating_df = avg_rating_df.rename(columns={'ratings': 'avg_ratings'})
avg_rating_df


,magazine_id,avg_ratings
0,B00005N7NQ,3.793103
1,B00005N7O3,3.600000
2,B00005N7O4,4.333333
3,B00005N7O6,4.531250
4,B00005N7O9,4.434783
...,...,...
2423,B01HI8V1AE,4.750000
2424,B01HI8V1C2,4.000000
2425,B01HI8V1I6,4.500000
2426,B01HI8V1MC,4.071429


In [15]:
popular_df = num_rating_df.merge(avg_rating_df, on = 'magazine_id')
popular_df.head()

,magazine_id,number_of_ratings,avg_ratings
0,B00005N7NQ,116,3.793103
1,B00005N7O3,5,3.600000
2,B00005N7O4,6,4.333333
3,B00005N7O6,64,4.531250
4,B00005N7O9,23,4.434783


In [16]:
popular_df.shape

(2428, 3)

In [17]:
popular_df = popular_df[popular_df['number_of_ratings']>=100].sort_values('avg_ratings', ascending=False).head(100)
popular_df.head()


,magazine_id,number_of_ratings,avg_ratings
1582,B0065MEDRI,343,4.781341
1314,B000NY15YI,109,4.761468
649,B00006LL1D,382,4.654450
1276,B000IOEJBO,144,4.645833
1344,B000UEI4JU,235,4.621277


In [18]:
popular_df.shape

(100, 3)

In [19]:
popular_df = popular_df.merge(df, on='magazine_id')
popular_df.shape

(33894, 5)

In [20]:
popular_df = popular_df.drop_duplicates('magazine_id')
popular_df.shape

(100, 5)

In [21]:
popular_df

,magazine_id,number_of_ratings,avg_ratings,user_id,ratings
0,B0065MEDRI,343,4.781341,A2TUZYARMUEPY2,5
343,B000NY15YI,109,4.761468,A2AX34ALJ2APN2,5
452,B00006LL1D,382,4.654450,A3NQI54DKBTTZ6,5
834,B000IOEJBO,144,4.645833,A24A2HVAN6784Z,5
978,B000UEI4JU,235,4.621277,A1YRODJA6SA0NS,5
...,...,...,...,...,...
30731,B000W3MB5M,441,4.031746,A3NWN21HH5CEWZ,5
31172,B000XXDJ70,109,4.027523,A5IBTO5P48ZNU,5
31281,B00005N7SM,362,4.022099,AC3OYGYW30MJB,5
31643,B000063XJL,772,3.996114,A1EMDSTJDUE6B0,5


In [22]:
popular_df = popular_df[['user_id', 'magazine_id', 'number_of_ratings', 'avg_ratings']]
popular_df.head()

,user_id,magazine_id,number_of_ratings,avg_ratings
0,A2TUZYARMUEPY2,B0065MEDRI,343,4.781341
343,A2AX34ALJ2APN2,B000NY15YI,109,4.761468
452,A3NQI54DKBTTZ6,B00006LL1D,382,4.654450
834,A24A2HVAN6784Z,B000IOEJBO,144,4.645833
978,A1YRODJA6SA0NS,B000UEI4JU,235,4.621277


# collaborative filtering apporach

In [23]:
#finding those user who have rated more than or equal to 10 magazine
#filtering based on users 
df_ind = df.groupby('user_id').count()['ratings']>10
user_that_rated_more_than_10 = df_ind[df_ind].index

In [24]:
df_user = df[df['user_id'].isin(user_that_rated_more_than_10)]
df_user

,user_id,magazine_id,ratings
2,A3JPFWKS83R49V,B00005N7OJ,3
73,A1RPTVW5VEOSI,B00005N7PS,1
246,AG6TX1ZJHLMO7,B00005N7PS,4
286,A1RPTVW5VEOSI,B00005N7OU,4
296,A3JPFWKS83R49V,B00005N7PS,3
...,...,...,...
88181,A3H8DSJE2TY9AU,B01EGFDTZW,4
88183,AC98CDA0GZDKJ,B01EGFDTZW,5
88276,A1J5ZHE8SG700J,B01H6WOLWC,5
88283,A1J5ZHE8SG700J,B01H6WOLSG,5


In [25]:
#filtering based on magazine
#filtering those 
b_ind = df_user.groupby('magazine_id').count()['ratings']>=4
famous_mag = b_ind[b_ind].index

In [26]:
df_final = df_user[df_user['magazine_id'].isin(famous_mag)]
df_final

,user_id,magazine_id,ratings
734,A2JJGIJI8X6KRS,B00005N7OV,5
737,A281NPSIMI1C2R,B00005N7OV,5
749,A281NPSIMI1C2R,B00005N7RD,5
1147,A1CKHSVHDVTKFT,B00005N7OV,5
1411,A1JZFGZEZVWQPY,B00005N7RD,4
...,...,...,...
75231,A254QZWPSXZG52,B000060MJC,5
79869,A3KCW7GFEL6V46,B00079RO7G,1
80946,A2OTUWUSH49XIN,B000ILY9LW,2
81069,A2LI10MAXV0815,B000IOE9Y6,5


In [27]:
df_final.drop_duplicates()

,user_id,magazine_id,ratings
734,A2JJGIJI8X6KRS,B00005N7OV,5
737,A281NPSIMI1C2R,B00005N7OV,5
749,A281NPSIMI1C2R,B00005N7RD,5
1147,A1CKHSVHDVTKFT,B00005N7OV,5
1411,A1JZFGZEZVWQPY,B00005N7RD,4
...,...,...,...
75231,A254QZWPSXZG52,B000060MJC,5
79869,A3KCW7GFEL6V46,B00079RO7G,1
80946,A2OTUWUSH49XIN,B000ILY9LW,2
81069,A2LI10MAXV0815,B000IOE9Y6,5


In [28]:
pt = df_final.pivot_table(index = 'magazine_id', columns = 'user_id', values= 'ratings')

In [29]:
pt

user_id,A10VAZLEGTEZU0,A1CKHSVHDVTKFT,A1EMDSTJDUE6B0,A1EWCCX0LSRO7J,A1FEDK55GMLSV,A1GQAKL9CGQLP1,A1HV7PFWWGX2K3,A1J5ZHE8SG700J,A1J6UA1MD96SKE,A1JZFGZEZVWQPY,...,AG6TX1ZJHLMO7,AGT43FXXJM1AP,AHD101501WCN1,AKMEY1BSHSDG7,AMKQP0R290TLS,AOR590VF8YI5J,AVF9FV7AMRP5C,AW3VZ5O895LRK,AXV3OJG9BZ2JW,AYFVLMPVVDC4L
magazine_id,,,,,,,,,,,,,,,,,,,,,
B00005N7OV,NaN,5.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00005N7PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00005N7PN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN
B00005N7Q1,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
B00005N7QA,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B001LF4EVO,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0
B0037STB02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
B00EVV77A0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [30]:
pt.fillna(0, inplace=True)

In [51]:
pt

user_id,A10VAZLEGTEZU0,A1CKHSVHDVTKFT,A1EMDSTJDUE6B0,A1EWCCX0LSRO7J,A1FEDK55GMLSV,A1GQAKL9CGQLP1,A1HV7PFWWGX2K3,A1J5ZHE8SG700J,A1J6UA1MD96SKE,A1JZFGZEZVWQPY,...,AG6TX1ZJHLMO7,AGT43FXXJM1AP,AHD101501WCN1,AKMEY1BSHSDG7,AMKQP0R290TLS,AOR590VF8YI5J,AVF9FV7AMRP5C,AW3VZ5O895LRK,AXV3OJG9BZ2JW,AYFVLMPVVDC4L
magazine_id,,,,,,,,,,,,,,,,,,,,,
B00005N7OV,0.0,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7PG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7PN,0.0,0.0,0.0,0.0,4.0,5.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0
B00005N7Q1,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
B00005N7QA,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B001LF4EVO,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0
B0037STB02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
B00EVV77A0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
# !pip install -U scikit-learn

In [38]:
similarity_score  = cosine_similarity(pt)

In [41]:
similarity_score

array([[1.        , 0.15362886, 0.29013059, ..., 0.40481612, 0.16922436,
        0.28429693],
       [0.15362886, 1.        , 0.11926968, ..., 0.        , 0.        ,
        0.        ],
       [0.29013059, 0.11926968, 1.        , ..., 0.25391425, 0.        ,
        0.16750598],
       ...,
       [0.40481612, 0.        , 0.25391425, ..., 1.        , 0.24310832,
        0.42300848],
       [0.16922436, 0.        , 0.        , ..., 0.24310832, 1.        ,
        0.16666667],
       [0.28429693, 0.        , 0.16750598, ..., 0.42300848, 0.16666667,
        1.        ]])

In [56]:
# helps to find the index of the magazine_id
np.where(pt.index == 'B01CF3ECNK')[0][0]


81

In [45]:
def recommend(magazine_name):
    #index fetch
    index = np.where(pt.index == magazine_name)[0][0]
    # sorting on the basis of score
    similar_item = sorted(list(enumerate(similarity_score[index])), key = lambda x:x[1], reverse=True)[1:6]
    for i in similar_item:
        print(pt.index[i[0]])

In [47]:
recommend('B00005N7QA')


B00005N7PG
B00005NIP7
B00005R8BA
B00005NIRE
B000085A6U


In [49]:
import pickle
# pickle.dump(popular_df, open('popular.pkl', 'wb'))

In [50]:
pickle.dump(similarity_score, open('similarity_score.pkl', 'wb'))


In [52]:
pickle.dump(pt, open('pivot_table.pkl', 'wb'))


## Recommending magazines as per rating

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


df = pd.read_csv("magazines_rating.csv.zip")
df.head()

,user_id,magazine_id,ratings
0,A3JPFWKS83R49V,B00005N7OJ,3
1,A5QQOOZJOVPSF,B00005N7P0,4
2,A26PEGXTHPZQEE,B00005N7P0,5
3,A5RHZE7B8SV5Q,B00005N7PS,5
4,A1RPTVW5VEOSI,B00005N7PS,1


In [10]:
# pivot table
pt = df.pivot_table(index = 'magazine_id', columns = 'user_id', values= 'ratings')
pivot_table = pt.fillna(0)
pivot_table

user_id,A10BWUA2MGA9BK,A10H24TDLK2VDP,A10PEXB6XAQ5XF,A10VAZLEGTEZU0,A10YB2NJ6X7C9U,A11O5WKA3VYLX1,A124ITVETU8BZA,A12H5SO3JZ4RE3,A12SMNXVNUCT29,A12UGM39O9MLC9,...,AXV3OJG9BZ2JW,AY6VA1SM227JF,AYAAX783KRCJZ,AYFVLMPVVDC4L,AYOMAHLWRQHUG,AZE7XAL1VV91,AZJ0TO1CVCQJT,AZKN6G4LKLQL2,AZSACV427GIAH,AZUK3G0CKWJZ1
magazine_id,,,,,,,,,,,,,,,,,,,,,
B00005N7NQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7O6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7O9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7OC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00005N7OD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01F2MKW0I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01HBMU68U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B01HI8V10E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
similarity_score  = cosine_similarity(pivot_table)

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.33425161],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.33425161, 0.        ,
        1.        ]])

In [35]:
def recommend_magazine(magazine_name):
    #index fetch
    index = np.where(pivot_table.index == magazine_name)[0][0]
    # sorting on the basis of score
    similar_item = sorted(list(enumerate(similarity_score[index])), key = lambda x:x[1], reverse=True)[1:10]
    items = [pivot_table.index[i[0]] for i in similar_item]
    return items

recommend_magazine("B00005NIRE")

['B00005NIO2',
 'B00005QJE2',
 'B00006KC3X',
 'B00006KSP0',
 'B00006LB29',
 'B00007J75K',
 'B0001IOZWK',
 'B000UHI334',
 'B00FP59VGY']

In [34]:
import pickle
pickle.dump(pivot_table, open('pivot_table.pkl', 'wb'))
pickle.dump(similarity_score, open('similarity_score.pkl', 'wb'))

## Using Nearest Neighbours

In [70]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


# create a sparse matrix of user-ratings
user_magazine_matrix = df.pivot_table(index='magazine_id', columns='user_id', values='ratings').fillna(0)
user_magazine_matrix_sparse = csr_matrix(user_magazine_matrix.values)
user_magazine_matrix_sparse

# train a nearest neighbors model on the sparse matrix
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_magazine_matrix_sparse)


# define a function to get the top N recommendations for a given magazine
def get_top_n_recommendations(magazine_id, N=5):
    # get the index of the magazine in the matrix
    magazine_index = user_magazine_matrix.index.get_loc(magazine_id)
    # find the k nearest neighbors of the magazine
    distances, indices = model_knn.kneighbors(user_magazine_matrix_sparse[magazine_index], n_neighbors=N+1)

    # get the indices of the k nearest neighbors (excluding the magazine itself)
    neighbor_indices = indices.flatten()[1:]

    # get the ratings of the k nearest neighbors for the input magazine
    neighbor_ratings = user_magazine_matrix.iloc[neighbor_indices]

    # compute the average rating of each magazine over the k nearest neighbors
    magazine_ratings = neighbor_ratings.mean(axis=1)

    # sort the magazines by average rating and return the top N
    top_magazines = magazine_ratings.sort_values(ascending=False)[:N]

    return top_magazines

# example usage: get the top 5 recommendations for user A3JPFWKS83R49V
print(get_top_n_recommendations('B01HBMU68U', N=5))

magazine_id
B001LF4EVO    0.472855
B00EZB37X2    0.201401
B00005N7T5    0.122592
B01HI8V10E    0.054291
B01HI8V1MC    0.010508
dtype: float64


## Singular Value Decomposition

In [76]:
# !pip install scikit-surprise

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from surprise import SVD, Dataset, Reader
from surprise.model_selection import GridSearchCV

# Load data from CSV
data = pd.read_csv('magazines_rating.csv.zip')

# Filter data to only include magazines with at least 5 ratings and users who have rated at least 10 magazines
magazine_counts = data['magazine_id'].value_counts()
user_counts = data['user_id'].value_counts()
data = data[data['magazine_id'].isin(magazine_counts.index[magazine_counts >= 5])]
data = data[data['user_id'].isin(user_counts.index[user_counts >= 10])]

# Split data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Create a surprise Dataset object
reader = Reader(rating_scale=(1, 5))
train_dataset = Dataset.load_from_df(train_data[['user_id', 'magazine_id', 'ratings']], reader)

# Use grid search to find the best hyperparameters for the SVD algorithm
param_grid = {'n_factors': [50, 100, 200], 'reg_all': [0.01, 0.1, 1]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(train_dataset)
# Print the best RMSE score and hyperparameters
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# Train the SVD algorithm on the full training dataset with the best hyperparameters
model = SVD(n_factors=gs.best_params['rmse']['n_factors'], reg_all=gs.best_params['rmse']['reg_all'])
model.fit(train_dataset.build_full_trainset())

# Make predictions on the test set and calculate RMSE
test_dataset = Dataset.load_from_df(test_data[['user_id', 'magazine_id', 'ratings']], reader)
test_predictions = model.test(test_dataset.build_full_trainset().build_testset())
print('Test RMSE:', mean_squared_error([pred.r_ui for pred in test_predictions], [pred.est for pred in test_predictions], squared=False))

# # Generate recommendations for a given user
# user_id = 'A3JPFWKS83R49V'
# user_items = data[data['user_id'] == user_id]['magazine_id'].unique()
# all_items = data['magazine_id'].unique()
# unrated_items = list(set(all_items) - set(user_items))
# testset = [[user_id, item, 4.] for item in unrated_items]
# predictions = model.test(testset)
# recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]
# recommended_items = [rec.iid for rec in recommendations]
# print('Recommended items:', recommended_items)

# Generate recommendations for a given magazine
magazine_id = "B000ILZ6VE" # replace with the magazine ID you want recommendations for
all_items = data['magazine_id'].unique()
unrated_items = list(set(all_items) - set(data[data['magazine_id'] == magazine_id]['magazine_id']))
testset = [[u, magazine_id, 4.] for u in unrated_items]
predictions = model.test(testset)
recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]
recommended_items = [rec.iid for rec in recommendations]
print('Recommended magazines for magazine', magazine_id, ':', recommended_items)


0.8988893398305365
{'n_factors': 50, 'reg_all': 0.1}
Test RMSE: 0.8494950119481627
Recommended magazines for magazine B000ILZ6VE : ['B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE', 'B000ILZ6VE']
